<a href="https://colab.research.google.com/github/kjiang25/Betting-Against-All-Betas/blob/main/Betting_Against_All_Betas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from math import prod

In [2]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Thesis/Code

Mounted at /content/drive
/content/drive/MyDrive/Thesis/Code


In [3]:
# Read stock data from Drive Folder
data = pd.read_csv("Full dataset.csv")
data['date'] = pd.to_datetime(data['date'])
data.rename(columns={'PERMNO': 'Permno'}, inplace=True)
data['date'] = data['date'] + pd.offsets.MonthEnd(0)
data['MktCap'] = data['PRC'] * data['SHROUT']

# Check for duplicates and remove if found
duplicates = data.groupby(['date', 'Permno']).size().reset_index(name='count')
duplicates = duplicates[duplicates['count'] > 1]

if not duplicates.empty:
  print("Found duplicate entries:")
  print(duplicates)

  returns = data.groupby(['date', 'Permno'])['RET'].mean().reset_index()

# Pivot stock data to returns dataframe
returns = data.pivot(index='date', columns='Permno', values='RET')
returns = returns.applymap(lambda x: pd.to_numeric(x, errors='coerce'))

# Display returns.head
returns.head()

<ipython-input-3-e47f1f14ee53>:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  returns = returns.applymap(lambda x: pd.to_numeric(x, errors='coerce'))


Permno,10000,10001,10002,10003,10005,10006,10007,10008,10009,10010,...,93427,93428,93429,93430,93431,93432,93433,93434,93435,93436
date,,,,,,,,,,,,,,,,,,,,,
1973-01-31,NaN,NaN,NaN,NaN,NaN,-0.060367,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1973-02-28,NaN,NaN,NaN,NaN,NaN,-0.075978,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1973-03-31,NaN,NaN,NaN,NaN,NaN,0.110429,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1973-04-30,NaN,NaN,NaN,NaN,NaN,-0.002762,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1973-05-31,NaN,NaN,NaN,NaN,NaN,-0.011634,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Read three factor file from Drive Folder
three_factors = pd.read_csv("Fama French 3 Factors.csv")
three_factors.columns = three_factors.iloc[2]
three_factors = three_factors.iloc[3:1181]
three_factors.rename(columns={three_factors.columns[0]: 'Date'}, inplace = True)
three_factors['Date'] = pd.to_datetime(three_factors['Date'], format = '%Y%m') + pd.offsets.MonthEnd(0)

# Read five factor file from Drive Folder
five_factors = pd.read_csv("Fama French 5 Factors.csv")
five_factors.columns = five_factors.iloc[2]
five_factors = five_factors.iloc[3:737]
five_factors.rename(columns={five_factors.columns[0]: 'Date'}, inplace = True)
five_factors['Date'] = pd.to_datetime(five_factors['Date'], format = '%Y%m') + pd.offsets.MonthEnd(0)

# Read momentum factor file from Drive Folder
Mom_factor = pd.read_csv("Fama French Momentum Factor.csv")
Mom_factor = Mom_factor.iloc[:1172]
Mom_factor['Date'] = pd.to_datetime(Mom_factor['Date'], format = '%Y%m') + pd.offsets.MonthEnd(0)

# Merge factors into one dataframe
factors = pd.merge(three_factors[['Date', 'Mkt-RF', 'SMB', 'HML']], Mom_factor[['Date', 'MOM']], on='Date')
factors = pd.merge(factors, five_factors[['Date', 'RMW', 'CMA', 'RF']], on='Date')
factors = factors.loc[(factors['Date'] >= pd.to_datetime('1973-01')) & (factors['Date'] <= pd.to_datetime('2024-01'))]
factors.set_index('Date', inplace = True)
factors = factors.astype(float)
factors /= 100

# Display factors.head
factors.head()

,Mkt-RF,SMB,HML,MOM,RMW,CMA,RF
Date,,,,,,,
1973-01-31,-0.0329,-0.0349,0.0268,0.0373,0.0042,0.0090,0.0044
1973-02-28,-0.0485,-0.0387,0.0160,0.0216,-0.0026,0.0002,0.0041
1973-03-31,-0.0130,-0.0282,0.0262,0.0359,-0.0107,0.0062,0.0046
1973-04-30,-0.0568,-0.0385,0.0541,0.0636,-0.0158,0.0260,0.0052
1973-05-31,-0.0294,-0.0630,0.0041,0.0714,0.0195,-0.0157,0.0051


In [5]:
# Create copy for excess returns
excess_returns = returns.copy()

# Subtract RF from all columns
excess_returns = excess_returns.sub(factors['RF'], axis=0)

# Fill na
excess_returns[returns.isna()] = returns[returns.isna()]

# Display excess_returns.head
excess_returns.head()

Permno,10000,10001,10002,10003,10005,10006,10007,10008,10009,10010,...,93427,93428,93429,93430,93431,93432,93433,93434,93435,93436
date,,,,,,,,,,,,,,,,,,,,,
1973-01-31,NaN,NaN,NaN,NaN,NaN,-0.064767,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1973-02-28,NaN,NaN,NaN,NaN,NaN,-0.080078,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1973-03-31,NaN,NaN,NaN,NaN,NaN,0.105829,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1973-04-30,NaN,NaN,NaN,NaN,NaN,-0.007962,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1973-05-31,NaN,NaN,NaN,NaN,NaN,-0.016734,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# Function to calculate deciles thresholds from 0 to 90
def calculate_deciles(data):
  deciles = []
  for i in range(0, 91, 10):
    decile = np.percentile(data, i)
    deciles.append(decile)
  return deciles

# Function to average the innermost values of nested lists, used to calculate average returns of group of stocks
def average_innermost_lists(nested_lists):

    if not isinstance(nested_lists, list):
        return nested_lists

    # Check if we're at the innermost list (contains numbers)
    if nested_lists and not isinstance(nested_lists[0], list):
        return np.mean(nested_lists)

    # Recursively process nested lists
    return [average_innermost_lists(sublist) for sublist in nested_lists]



In [8]:
# Initialize date window for regressions
window_start = pd.to_datetime('1973-01-31')
window_end = window_start + pd.DateOffset(years=5) - pd.offsets.MonthEnd(1)
final_date = pd.to_datetime(returns.index[-1])

date_index = pd.date_range(start=window_end + pd.offsets.MonthEnd(1), end=final_date, freq='M')

# Initialize lists to store decile returns
decile_returns = []

# Select factor to analyze
factor = 'Mkt-RF'

# Run regressions with monthly frequency
while window_end < final_date:

  # Get top 500 assets at the end of each calendar year
  next_year = window_end + pd.DateOffset(years=1)
  df_date = data[data['date'] == window_end]
  top_assets = df_date.sort_values('MktCap', ascending = False).head(50) # Replace with 500, takes roughly 15 minutes to run
  permnos = top_assets['Permno'].tolist()

  # Monthly loop
  while window_end < next_year and window_end < final_date:

    # Initialize lists to store decile buckets and beta+returns for month
    decile_returns_row = [[] for i in range(10)]
    betas_returns = []

    # Iterate through assets
    for permno in permnos:

      # Filter data by regression window
      filtered_returns = excess_returns.loc[window_start:window_end]
      filtered_factors = factors.loc[window_start:window_end]

      # Run regression
      if filtered_returns[permno].notnull().sum() / len(filtered_returns )> 0.5 and filtered_returns[permno][-12:].notnull().all(): # at least 50% of returns, most recent year filled
        regression = sm.OLS(filtered_returns[permno], sm.add_constant(filtered_factors[factor]),missing='drop').fit() # Calculate speficied factor beta
        beta = regression.params[factor] # Get beta
        next_month_excess_return = excess_returns.loc[window_end + pd.offsets.MonthEnd(1), permno] # Get next month excess return of that asset

        # Store beta and next month excess return pair into list if next month return is not NaN
        if not np.isnan(next_month_excess_return):
          br_entry = [beta, next_month_excess_return]
          betas_returns.append(br_entry)

    # Calculate beta decile thresholds
    betas = [item[0] for item in betas_returns]
    deciles = calculate_deciles(betas)

    # Iterate through beta+returns pairs and append next month returns to corresponding decile bucket
    for i in range(len(betas_returns)):
      for d in range(9, -1, -1): # Iterate backwards from high beta decile to low beta decile
        if betas_returns[i][0] >= deciles[d]: # If beta is greater than or equal to decile threshold
          decile_returns_row[9-d].append(betas_returns[i][1]) # Append next month return to corresponding decile bucket, where decile 0 is the highest beta decile and decile 9 is the lowest beta decile
          break

    decile_returns_row = average_innermost_lists(decile_returns_row) # Calculate average excess next month returns for each decile
    decile_returns.append(decile_returns_row) # Append decile returns to list

    # Increment dates by one month
    window_start += pd.offsets.MonthEnd(1)
    window_end += pd.offsets.MonthEnd(1)


decile_returnsdf = pd.DataFrame(decile_returns, columns = [i for i in range(1, 11)], index = date_index) # Convert nested decile returns list to dataframe
decile_returnsdf.head()

<ipython-input-8-d7f1c5d40ff5>:6: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  date_index = pd.date_range(start=window_end + pd.offsets.MonthEnd(1), end=final_date, freq='M')


,1,2,3,4,5,6,7,8,9,10
1978-01-31,-0.079567,-0.080514,-0.063314,-0.079874,-0.067906,-0.050404,-0.068083,-0.077988,-0.079294,-0.065178
1978-02-28,-0.024387,-0.036124,-0.065798,-0.041426,-0.036126,-0.013645,-0.023999,-0.011963,-0.014063,-0.014973
1978-03-31,0.032573,0.049508,-0.003300,0.031400,-0.024504,0.029968,0.001903,0.009433,0.030294,0.019850
1978-04-30,0.136993,0.114154,0.134624,0.118304,0.129803,0.046957,0.071008,0.083964,0.064259,0.048694
1978-05-31,-0.009330,0.016880,-0.016723,0.000395,0.010253,-0.010507,0.009071,-0.015413,0.016594,-0.001541


In [9]:
# Initialize regression window
regression_start = pd.to_datetime('1978-01-31')
regression_end = pd.to_datetime(decile_returnsdf.index[-1])

# Initialize empty factor neutral decile returns dataframe and lists to store data
factor_neutral_decile_returnsdf = pd.DataFrame(columns = [i for i in range(1, 11)], index = decile_returnsdf.index)
alphas = [[] for i in range(10)]
factor_betas = [[] for i in range(10)]
r_squared = [[] for i in range(10)]

# Iterate through each decile's returns
for i in range(10):

  # Initialize lists to store betas
  factor_beta_entries = []

  # Filter factor data
  filtered_factors = factors.loc[regression_start:regression_end, ['Mkt-RF']] # CAPM specification
  # filtered_factors = factors.loc[regression_start:regression_end, ['Mkt-RF', 'SMB', 'HML']] # 3-Factor specification
  # filtered_factors = factors.loc[regression_start:regression_end, ['Mkt-RF', 'SMB', 'HML', 'MOM]] # 4-Factor specification
  # filtered_factors = factors.loc[regression_start:regression_end, ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']] # 5-Factor specification

  regression = sm.OLS(decile_returnsdf[i+1], sm.add_constant(filtered_factors), missing='drop').fit() # Run regression

  mkt_beta = regression.params['Mkt-RF'] # Assign beta value
  # smb_beta = regression.params['SMB'] # Add if necessary
  # hml_beta = regression.params['HML']
  # mom_beta = regression.params['MOM']
  # rmw_beta = regression.params['RMW']
  # cma_beta = regression.params['CMA']
  r_squared[i] = regression.rsquared # Store R-squared value
  alphas[i] = regression.params['const'] # Store alpha value

  # Append factor betas to list
  factor_beta_entries.append(mkt_beta)
  # factor_beta_entries.append(smb_beta) # Add if necessary
  # factor_beta_entries.append(hml_beta)
  # factor_beta_entries.append(rmw_beta)
  # factor_beta_entries.append(cma_beta)

  # Calculate factor neutral decile returns equal to alpha of regression
  factor_neutral_decile_returnsdf[i+1] = decile_returnsdf[i+1] - mkt_beta * filtered_factors['Mkt-RF'] # CAPM specification
  # factor_neutral_decile_returnsdf[i+1] = decile_returnsdf[i+1] - mkt_beta * filtered_factors['Mkt-RF'] - smb_beta * filtered_factors['SMB'] - hml_beta * filtered_factors['HML'] # 3-Factor specification
  # factor_neutral_decile_returnsdf[i+1] = decile_returnsdf[i+1] - mkt_beta * filtered_factors['Mkt-RF'] - smb_beta * filtered_factors['SMB'] - hml_beta * filtered_factors['HML'] - mom_beta * filtered_factors['MOM'] # 4-Factor specification
  # factor_neutral_decile_returnsdf[i+1] = decile_returnsdf[i+1] - mkt_beta * filtered_factors['Mkt-RF'] - smb_beta * filtered_factors['SMB'] - hml_beta * filtered_factors['HML'] - rmw_beta * filtered_factors['RMW'] - cma_beta * filtered_factors['CMA'] # 5-Factor specification

  # Append factor betas to list
  factor_betas[i] = factor_beta_entries

# Convert r_squared list to dataframe
r_squareddf = pd.DataFrame(r_squared).T
r_squareddf.columns = [i for i in range(1, 11)]

# Convert alphas list to dataframe
alphasdf = pd.DataFrame(alphas).T
alphasdf.columns = [i for i in range(1, 11)]

# Convert factor_betas list to dataframe
factor_betasdf = pd.DataFrame(factor_betas).T
factor_betasdf.columns = [i for i in range(1, 11)]

In [10]:
# Calculate average factor neutral returns for each decile
returns_avg = pd.DataFrame([factor_neutral_decile_returnsdf.mean()])
returns_avg

,1,2,3,4,5,6,7,8,9,10
0,-0.003885,-0.004094,-0.002237,0.000363,-1.452767e-07,-0.000284,0.003063,0.002879,0.001036,0.003434


In [12]:
# Initialize list to store all performance metrics
performance_metrics = []

# Iterate through each decile and calculate performance metrics
for i in range(10):

  # Initialize list to store performance metrics for each decile
  entry = []
  monthly_arithmetic_mean = factor_neutral_decile_returnsdf[i+1].mean() # Calculate monthly arithmetic mean
  annual_arithmetic_mean = monthly_arithmetic_mean * 12 # Calculate annual arithmetic mean
  monthly_volatility = factor_neutral_decile_returnsdf[i+1].std() # Calculate monthly volatility
  annual_volatility = monthly_volatility * np.sqrt(12) # Calculate annual volatility
  monthly_geometric_mean = (1 + factor_neutral_decile_returnsdf[i+1]).prod() ** (1/len(factor_neutral_decile_returnsdf)) - 1 # Calculate monthly geometric mean
  annual_geometric_mean = (1 + monthly_geometric_mean) ** 12 - 1 # Calculate annual geometric mean
  sharpe_ratio = annual_arithmetic_mean / annual_volatility # Calculate Sharpe ratio
  t_stat = (np.sqrt(len(factor_neutral_decile_returnsdf)) * sharpe_ratio) / np.sqrt(12) # Calculate t-statistic

  # Append performance metrics to list
  entry.append(annual_arithmetic_mean)
  entry.append(t_stat)
  entry.append(annual_geometric_mean)
  entry.append(annual_volatility)
  entry.append(sharpe_ratio)
  performance_metrics.append(entry)

index = [i for i in range(1,11)] # Assign index values for deciles
columns = ['Arithmetic Mean', 't-stat', 'Geometric Mean', 'Volatility', 'Sharpe Ratio'] # Assign column names for performance metrics
performance_metricsdf = pd.DataFrame(performance_metrics, columns = columns, index = index).T # Convert performance metrics list to dataframe
performance_metricsdf


,1,2,3,4,5,6,7,8,9,10
Arithmetic Mean,-0.046618,-0.049126,-0.026843,0.004354,-0.000002,-0.003405,0.036750,0.034544,0.012426,0.041204
t-stat,-2.033032,-2.534783,-1.502164,0.265727,-0.000116,-0.245590,2.186110,2.153231,0.715086,2.582282
Geometric Mean,-0.057187,-0.056316,-0.033691,-0.001808,-0.005231,-0.007787,0.030703,0.029043,0.005514,0.035941
Volatility,0.155519,0.131446,0.121198,0.111132,0.102320,0.094040,0.114017,0.108809,0.117858,0.108222
Sharpe Ratio,-0.299754,-0.373733,-0.221482,0.039179,-0.000017,-0.036210,0.322324,0.317477,0.105434,0.380737


In [14]:
# Create betting against beta portfolio that goes long the lowest beta decile and short the highest beta decile
betting_against_betadf = pd.DataFrame(factor_neutral_decile_returnsdf[10] - factor_neutral_decile_returnsdf[1], columns=['BAB Return'])

# Calculate performance metrics for betting against beta portfolio
monthly_arithmetic_mean = betting_against_betadf['BAB Return'].mean() # Calculate monthly arithmetic mean
annual_arithmetic_mean = monthly_arithmetic_mean * 12 # Calculate annual arithmetic mean
monthly_volatility = betting_against_betadf['BAB Return'].std() # Calculate monthly volatility
annual_volatility = monthly_volatility * np.sqrt(12) # Calculate annual volatility
monthly_geometric_mean = (1 + betting_against_betadf['BAB Return']).prod() ** (1/len(factor_neutral_decile_returnsdf)) - 1 # Calculate monthly geometric mean
annual_geometric_mean = (1 + monthly_geometric_mean) ** 12 - 1 # Calculate annual geometric mean
sharpe_ratio = annual_arithmetic_mean / annual_volatility # Calculate Sharpe ratio
t_stat = (np.sqrt(len(factor_neutral_decile_returnsdf)) * sharpe_ratio) / np.sqrt(12) # Calculate t-statistic
print('Arithmetic Mean: ', annual_arithmetic_mean)
print('t-stat: ', t_stat)
print('Geometric Mean: ', annual_geometric_mean)
print('Volatility: ', annual_volatility)
print('Sharpe Ratio: ', sharpe_ratio)


Arithmetic Mean:  0.0878217507865045
t-stat:  2.7951475974373396
Geometric Mean:  0.06701985187833515
Volatility:  0.21309647264278941
Sharpe Ratio:  0.4121220295078223
